# Apache Hudi Core Conceptions (5) - COW: Clustering

## 1. Configuration

In [1]:
%%sh
# deploy hudi bundle jar
hdfs dfs -copyFromLocal -f /usr/lib/hudi/hudi-spark-bundle.jar /tmp/hudi-spark-bundle.jar
hdfs dfs -ls /tmp/hudi-spark-bundle.jar

-rw-r--r--   1 emr-notebook hdfsadmingroup   61421977 2023-03-19 02:27 /tmp/hudi-spark-bundle.jar


In [2]:
%%configure -f
{
    "conf" : {
        "spark.jars":"hdfs:///tmp/hudi-spark-bundle.jar",            
        "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
        "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.hudi.catalog.HoodieCatalog"
    }
}

In [3]:
%env S3_BUCKET=apache-hudi-core-conceptions

env: S3_BUCKET=apache-hudi-core-conceptions


In [4]:
%%sql
set S3_BUCKET=apache-hudi-core-conceptions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
94,application_1678096020253_0180,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%env WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions

env: WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions


In [6]:
%%sh
# make workspace
mkdir -p $WORKSPACE
# deploy hudi-stat.sh, a utility shell script to output hudi table status
wget https://raw.githubusercontent.com/bluishglc/apache-hudi-core-conceptions/master/hudi-stat.sh -O $WORKSPACE/hudi-stat.sh &>/dev/null
chmod a+x $WORKSPACE/hudi-stat.sh
ls $WORKSPACE/hudi-stat.sh

/home/emr-notebook/apache-hudi-core-conceptions/hudi-stat.sh


In [7]:
%%html
<style>
table {float:left}
</style>

## 2. Test Case 1 - Sync Clustering ( Inline Schedule + Inline Execute )

### 2.1. Test Plan

Step No.|Action|Volume / Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Small File
2|Insert|213MB|+1 Max File +1 Small File
3|Insert|182MB|+1 Max File +1 Small File

### 2.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.clustering.inline|false|true
hoodie.clustering.schedule.inline|false|false
hoodie.clustering.async.enabled|false|false
hoodie.clustering.inline.max.commits|4|3
hoodie.clustering.plan.strategy.target.file.max.bytes|1073741824 / 1GB|314572800 / 300MB
hoodie.clustering.plan.strategy.small.file.limit|314572800 / 300MB|209715200 / 200MB
hoodie.clustering.plan.strategy.sort.columns|-|review_date
hoodie.parquet.small.file.limit|104857600 / 100MB | 0
hoodie.copyonwrite.record.size.estimate|1024|175

### 2.3. Set Variables

In [8]:
%%sql
set TABLE_NAME=reviews_cow_clustering_1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%env TABLE_NAME=reviews_cow_clustering_1

env: TABLE_NAME=reviews_cow_clustering_1


### 2.4. Create Table

In [10]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [11]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating int, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.clustering.inline = 'true',
    hoodie.clustering.schedule.inline = 'false',
    hoodie.clustering.async.enabled = 'false',
    hoodie.clustering.inline.max.commits = '3',
    hoodie.clustering.plan.strategy.small.file.limit = '209715200',
    hoodie.clustering.plan.strategy.target.file.max.bytes = '314572800',
    hoodie.clustering.plan.strategy.sort.columns = 'review_date',
    hoodie.parquet.small.file.limit = '0',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 2.5. Insert 96MB / +1 Small File

In [13]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319022828271 │ commit │ COMPLETED │ 03-19 02:28 │ 03-19 02:28 │ 03-19 02:29 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪═════════════════════╪════════════════

### 2.6. Insert 213MB / +1 Max File +1 Small File

In [15]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year in (2004, 2005);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319022828271 │ commit │ COMPLETED │ 03-19 02:28 │ 03-19 02:28 │ 03-19 02:29 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319022943245 │ commit │ COMPLETED │ 03-19 02:29 │ 03-19 02:30 │ 03-19 02:30 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Adde

### 2.7. Insert 182MB / +1 Max File +1 Small File

In [17]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2007;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [18]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319022828271 │ commit        │ COMPLETED │ 03-19 02:28 │ 03-19 02:28 │ 03-19 02:29 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319022943245 │ commit        │ COMPLETED │ 03-19 02:29 │ 03-19 02:30 │ 03-19 02:30 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319023107794 │ commit        │ COMPLETED │ 03-19 02:31 │ 03-19 02:31 │ 03-19 02:32 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

## 3. Test Case 2 - Async Clustering ( Offline Schedule -> Offline Execute )

### 3.1. Test Plan

Step No.|Action|Volume / Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Small File
2|Insert|213MB|+1 Max File +1 Small File
3|Insert|182MB|+1 Max File +1 Small File
4|Offline Schedule|N/A|N/A
5|Offline Eexecute|491MB|+2 Clustered Files

### 3.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.clustering.inline|false|false
hoodie.clustering.schedule.inline|false|false
hoodie.clustering.async.enabled|false|true
hoodie.clustering.async.max.commits|4|3
hoodie.clustering.plan.strategy.target.file.max.bytes|1073741824 / 1GB|314572800 / 300MB
hoodie.clustering.plan.strategy.small.file.limit|314572800 / 300MB|209715200 / 200MB
hoodie.clustering.plan.strategy.sort.columns|-|review_date
hoodie.parquet.small.file.limit|104857600 / 100MB | 0
hoodie.copyonwrite.record.size.estimate|1024|175

### 3.3. Set Variables

In [19]:
%%sql
set TABLE_NAME=reviews_cow_clustering_2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%env TABLE_NAME=reviews_cow_clustering_2

env: TABLE_NAME=reviews_cow_clustering_2


### 3.4. Create Table

In [21]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [22]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [23]:
%%sql
-- for async mode: 
-- 1. some clustering properties works as table properties, i.e. hoodie.clustering.async.enabled and hoodie.clustering.async.max.commits
-- 2. all clustering async & plan properties will be overwritten by spark job level configuration via --hoodie-conf or --props
-- so, to be simple, do NOT set any clustering-ralated properties on table, always set by --hoodie-conf or --props
-- so, comment out clustering related settings, just keep them as reference.
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating int, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    -- hoodie.clustering.inline = 'false',
    -- hoodie.clustering.schedule.inline = 'false',
    -- hoodie.clustering.async.enabled = 'true',
    -- hoodie.clustering.async.max.commits = '3',
    -- hoodie.clustering.plan.strategy.small.file.limit = '209715200',
    -- hoodie.clustering.plan.strategy.target.file.max.bytes = '314572800',
    -- hoodie.clustering.plan.strategy.sort.columns = 'review_date',
    hoodie.parquet.small.file.limit = '0',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 3.5. Insert 96MB / +1 Small File

In [24]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [25]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319023511699 │ commit │ COMPLETED │ 03-19 02:35 │ 03-19 02:35 │ 03-19 02:35 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪═════════════════════╪════════════════

### 3.6. Insert 213MB / +1 Max File +1 Small File

In [26]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year in (2004, 2005);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [27]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319023511699 │ commit │ COMPLETED │ 03-19 02:35 │ 03-19 02:35 │ 03-19 02:35 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319023610998 │ commit │ COMPLETED │ 03-19 02:36 │ 03-19 02:36 │ 03-19 02:37 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Adde

### 3.7. Insert 182MB / +1 Max File +1 Small File

In [28]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2007;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [29]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319023511699 │ commit │ COMPLETED │ 03-19 02:35 │ 03-19 02:35 │ 03-19 02:35 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319023610998 │ commit │ COMPLETED │ 03-19 02:36 │ 03-19 02:36 │ 03-19 02:37 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319023729768 │ commit │ COMPLETED │ 03-19 02:37 │ 03-19 02:37 │ 03-19 02:38 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤══════════════════

### 3.8. Offline Schedule

In [30]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieClusteringJob' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'schedule' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "${TABLE_NAME}" \
  --hoodie-conf "hoodie.clustering.async.enabled=true" \
  --hoodie-conf "hoodie.clustering.async.max.commits=3" \
  --hoodie-conf "hoodie.clustering.plan.strategy.small.file.limit=209715200" \
  --hoodie-conf "hoodie.clustering.plan.strategy.target.file.max.bytes=314572800" \
  --hoodie-conf "hoodie.clustering.plan.strategy.sort.columns=review_date" > ${WORKSPACE}/${TABLE_NAME}.schedule.out &>/dev/null

In [31]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319023511699 │ commit        │ COMPLETED │ 03-19 02:35 │ 03-19 02:35 │ 03-19 02:35 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319023610998 │ commit        │ COMPLETED │ 03-19 02:36 │ 03-19 02:36 │ 03-19 02:37 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319023729768 │ commit        │ COMPLETED │ 03-19 02:37 │ 03-19 02:37 │ 03-19 02:38 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

### 3.9. Offline Eexecute 491MB / +2 Clustered Files

In [32]:
%%sh
# no need to provide --hoodie-conf setting again for execute, they are only used for schedule
# for execute, if there are requested replacecommits in timeline, they will be executed via this job.
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieClusteringJob' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "${TABLE_NAME}" > ${WORKSPACE}/${TABLE_NAME}.execute.out &>/dev/null

In [33]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319023511699 │ commit        │ COMPLETED │ 03-19 02:35 │ 03-19 02:35 │ 03-19 02:35 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319023610998 │ commit        │ COMPLETED │ 03-19 02:36 │ 03-19 02:36 │ 03-19 02:37 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319023729768 │ commit        │ COMPLETED │ 03-19 02:37 │ 03-19 02:37 │ 03-19 02:38 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

## 4. Test Case 3 - Async Clustering ( Offline Schedule，Offline Execute )

### 4.1. Test Plan

Step No.|Action|Volume / Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Small File
2|Insert|213MB|+1 Max File +1 Small File
3|Insert|182MB|+1 Max File +1 Small File
4|Offline Schedule + Eexecute|491MB|+2 Clustered Files

### 4.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.clustering.inline|false|false
hoodie.clustering.schedule.inline|false|false
hoodie.clustering.async.enabled|false|true
hoodie.clustering.async.max.commits|4|3
hoodie.clustering.plan.strategy.target.file.max.bytes|1073741824 / 1GB|314572800 / 300MB
hoodie.clustering.plan.strategy.small.file.limit|314572800 / 300MB|209715200 / 200MB
hoodie.clustering.plan.strategy.sort.columns|-|review_date
hoodie.parquet.small.file.limit|104857600 / 100MB | 0
hoodie.copyonwrite.record.size.estimate|1024|175

### 4.3. Set Variables

In [34]:
%%sql
set TABLE_NAME=reviews_cow_clustering_3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [35]:
%env TABLE_NAME=reviews_cow_clustering_3

env: TABLE_NAME=reviews_cow_clustering_3


### 4.4. Create Table

In [36]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [37]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [38]:
%%sql
-- for async mode: 
-- 1. some clustering properties works as table properties, i.e. hoodie.clustering.async.enabled and hoodie.clustering.async.max.commits
-- 2. all clustering async & plan properties will be overwritten by spark job level configuration via --hoodie-conf or --props
-- so, to be simple, do NOT set any clustering-ralated properties on table, always set by --hoodie-conf or --props
-- so, comment out clustering related settings, just keep them as reference.
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating int, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    -- hoodie.clustering.inline = 'false',
    -- hoodie.clustering.schedule.inline = 'false',
    -- hoodie.clustering.async.enabled = 'true',
    -- hoodie.clustering.async.max.commits = '3',
    -- hoodie.clustering.plan.strategy.small.file.limit = '209715200',
    -- hoodie.clustering.plan.strategy.target.file.max.bytes = '314572800',
    -- hoodie.clustering.plan.strategy.sort.columns = 'review_date',
    hoodie.parquet.small.file.limit = '0',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 4.5. Insert 96MB / +1 Small File

In [39]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [40]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319024252739 │ commit │ COMPLETED │ 03-19 02:42 │ 03-19 02:43 │ 03-19 02:43 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪═════════════════════╪════════════════

### 4.6. Insert 213MB / +1 Max File +1 Small File

In [41]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year in (2004, 2005);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [42]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319024252739 │ commit │ COMPLETED │ 03-19 02:42 │ 03-19 02:43 │ 03-19 02:43 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319024406012 │ commit │ COMPLETED │ 03-19 02:44 │ 03-19 02:44 │ 03-19 02:45 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Adde

### 4.7. Insert 182MB / +1 Max File +1 Small File

In [43]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2007;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [44]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319024252739 │ commit │ COMPLETED │ 03-19 02:42 │ 03-19 02:43 │ 03-19 02:43 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319024406012 │ commit │ COMPLETED │ 03-19 02:44 │ 03-19 02:44 │ 03-19 02:45 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319024531489 │ commit │ COMPLETED │ 03-19 02:45 │ 03-19 02:45 │ 03-19 02:46 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤══════════════════

### 4.8. Offline Schedule + Eexecute 491MB / +2 Clustered Files

In [45]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieClusteringJob' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'scheduleAndExecute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "${TABLE_NAME}" \
  --hoodie-conf "hoodie.clustering.async.enabled=true" \
  --hoodie-conf "hoodie.clustering.async.max.commits=3" \
  --hoodie-conf "hoodie.clustering.plan.strategy.small.file.limit=209715200" \
  --hoodie-conf "hoodie.clustering.plan.strategy.target.file.max.bytes=314572800" \
  --hoodie-conf "hoodie.clustering.plan.strategy.sort.columns=review_date" > ${WORKSPACE}/${TABLE_NAME}.schedule.out &>/dev/null

In [46]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319024252739 │ commit        │ COMPLETED │ 03-19 02:42 │ 03-19 02:43 │ 03-19 02:43 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319024406012 │ commit        │ COMPLETED │ 03-19 02:44 │ 03-19 02:44 │ 03-19 02:45 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319024531489 │ commit        │ COMPLETED │ 03-19 02:45 │ 03-19 02:45 │ 03-19 02:46 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

## 5. Test Case 4 - Semi Async Clustering ( Inline Schedule，Offline Execute )

NOTE:

This mode has a problem, under this mode, both `hoodie.clustering.inline.max.commits` or `hoodie.clustering.async.max.commits` does NOT work. This behavior is defferent from comapcation

### 5.1. Test Plan

Step No.|Action|Volume / Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Small File
2|Insert|213MB|+1 Max File +1 Small File
3|Insert|182MB|+1 Max File +1 Small File
4|Offline Eexecute|96MB|+1 Clustered File
5|Insert|14.6MB|+1 Small File
6|Offline Eexecute|503MB|+2 Clustered Files

### 5.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.clustering.inline|false|false
hoodie.clustering.schedule.inline|false|true
hoodie.clustering.async.enabled|false|false
hoodie.clustering.plan.strategy.target.file.max.bytes|1073741824 / 1GB|314572800 / 300MB
hoodie.clustering.plan.strategy.small.file.limit|314572800 / 300MB|209715200 / 200MB
hoodie.clustering.plan.strategy.sort.columns|-|review_date
hoodie.parquet.small.file.limit|104857600 / 100MB | 0
hoodie.copyonwrite.record.size.estimate|1024|175

### 5.3. Set Variables

In [47]:
%%sql
set TABLE_NAME=reviews_cow_clustering_4

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [48]:
%env TABLE_NAME=reviews_cow_clustering_4

env: TABLE_NAME=reviews_cow_clustering_4


### 5.4. Create Table

In [49]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [50]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [51]:
%%sql
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating int, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.clustering.inline = 'false',
    hoodie.clustering.schedule.inline = 'true',
    hoodie.clustering.async.enabled = 'false',
    -- NOT work for this mode
    hoodie.clustering.inline.max.commits = '3',
    -- NOT work for this mode
    hoodie.clustering.async.max.commits = '3',
    hoodie.clustering.plan.strategy.small.file.limit = '209715200',
    hoodie.clustering.plan.strategy.target.file.max.bytes = '314572800',
    hoodie.clustering.plan.strategy.sort.columns = 'review_date',
    hoodie.parquet.small.file.limit = '0',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 5.5. Insert 96MB / +1 Small File

In [52]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [53]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319025028437 │ commit        │ COMPLETED │ 03-19 02:50 │ 03-19 02:50 │ 03-19 02:51 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319025121426 │ replacecommit │ REQUESTED │ 03-19 02:51 │ -           │ -           ║
╚═════╧═══════════════════╧═══════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ Co

### 5.6. Insert 213MB / +1 Max File +1 Small File

In [54]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year in (2004, 2005);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [55]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319025028437 │ commit        │ COMPLETED │ 03-19 02:50 │ 03-19 02:50 │ 03-19 02:51 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319025121426 │ replacecommit │ REQUESTED │ 03-19 02:51 │ -           │ -           ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319025144123 │ commit        │ COMPLETED │ 03-19 02:51 │ 03-19 02:52 │ 03-19 02:52 ║
╚═════╧═══════════════════╧═══════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ CO

### 5.7. Insert 182MB / +1 Max File +1 Small File

In [56]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2007;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [57]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319025028437 │ commit        │ COMPLETED │ 03-19 02:50 │ 03-19 02:50 │ 03-19 02:51 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319025121426 │ replacecommit │ REQUESTED │ 03-19 02:51 │ -           │ -           ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319025144123 │ commit        │ COMPLETED │ 03-19 02:51 │ 03-19 02:52 │ 03-19 02:52 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

### 5.8. Offline Eexecute 96MB / +1 Clustered File

In [58]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieClusteringJob' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "${TABLE_NAME}" > ${WORKSPACE}/${TABLE_NAME}.execute.out &>/dev/null

In [59]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319025028437 │ commit        │ COMPLETED │ 03-19 02:50 │ 03-19 02:50 │ 03-19 02:51 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319025121426 │ replacecommit │ COMPLETED │ 03-19 02:51 │ 03-19 02:54 │ 03-19 02:55 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319025144123 │ commit        │ COMPLETED │ 03-19 02:51 │ 03-19 02:52 │ 03-19 02:52 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

### 5.9. Insert 14.6MB / +1 Small File

In [60]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 1998;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [61]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319025028437 │ commit        │ COMPLETED │ 03-19 02:50 │ 03-19 02:50 │ 03-19 02:51 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319025121426 │ replacecommit │ COMPLETED │ 03-19 02:51 │ 03-19 02:54 │ 03-19 02:55 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319025144123 │ commit        │ COMPLETED │ 03-19 02:51 │ 03-19 02:52 │ 03-19 02:52 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  

### 5.10. Offline Eexecute 503MB / +2 Clustered Files

In [62]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieClusteringJob' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "${TABLE_NAME}" > ${WORKSPACE}/${TABLE_NAME}.execute.out &>/dev/null

In [63]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤═══════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action        │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │               │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═══════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230319025028437 │ commit        │ COMPLETED │ 03-19 02:50 │ 03-19 02:50 │ 03-19 02:51 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230319025121426 │ replacecommit │ COMPLETED │ 03-19 02:51 │ 03-19 02:54 │ 03-19 02:55 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230319025144123 │ commit        │ COMPLETED │ 03-19 02:51 │ 03-19 02:52 │ 03-19 02:52 ║
╟─────┼───────────────────┼───────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3  